<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Automated-downloading" data-toc-modified-id="Automated-downloading-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Automated downloading</a></span><ul class="toc-item"><li><span><a href="#Javascript-codes" data-toc-modified-id="Javascript-codes-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Javascript codes</a></span></li><li><span><a href="#Downloading-Google-Images-using-python" data-toc-modified-id="Downloading-Google-Images-using-python-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Downloading Google Images using python</a></span></li><li><span><a href="#Detect-and-remove-duplicate-images-from-a-dataset-for-deep-learning" data-toc-modified-id="Detect-and-remove-duplicate-images-from-a-dataset-for-deep-learning-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Detect and remove duplicate images from a dataset for deep learning</a></span><ul class="toc-item"><li><span><a href="#Image-duplicate-detector" data-toc-modified-id="Image-duplicate-detector-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Image duplicate detector</a></span></li></ul></li></ul></li></ul></div>

# Automated downloading 

1. Open chrome browser
2. Search images to download
3. Goto image section
4. Right click on image section and click inspect 
5. Goto console
6. go back to image search section and scroll down the page until all images on the page has been displayed
7. repeat step 5
8. Copy and paste the javascript codes sections and hit enter on the console until a url text file is downloaded.

## Javascript codes

In [ ]:
/**
 * simulate a right-click event so we can grab the image URL using the
 * context menu alleviating the need to navigate to another page
 *
 * attributed to @jmiserez: http://pyimg.co/9qe7y
 *
 * @param   {object}  element  DOM Element
 *
 * @return  {void}
 */
# Copy the below code on console and hit enter
function simulateRightClick( element ) {
    var event1 = new MouseEvent( 'mousedown', {
        bubbles: true,
        cancelable: false,
        view: window,
        button: 2,
        buttons: 2,
        clientX: element.getBoundingClientRect().x,
        clientY: element.getBoundingClientRect().y
    } );
    element.dispatchEvent( event1 );
    var event2 = new MouseEvent( 'mouseup', {
        bubbles: true,
        cancelable: false,
        view: window,
        button: 2,
        buttons: 0,
        clientX: element.getBoundingClientRect().x,
        clientY: element.getBoundingClientRect().y
    } );
    element.dispatchEvent( event2 );
    var event3 = new MouseEvent( 'contextmenu', {
        bubbles: true,
        cancelable: false,
        view: window,
        button: 2,
        buttons: 0,
        clientX: element.getBoundingClientRect().x,
        clientY: element.getBoundingClientRect().y
    } );
    element.dispatchEvent( event3 );
}

In [ ]:
/**
 * grabs a URL Parameter from a query string because Google Images
 * stores the full image URL in a query parameter
 *
 * @param   {string}  queryString  The Query String
 * @param   {string}  key          The key to grab a value for
 *
 * @return  {string}               value
 */

# Copy the below code on console and hit enter
function getURLParam( queryString, key ) {
    var vars = queryString.replace( /^\?/, '' ).split( '&' );
    for ( let i = 0; i < vars.length; i++ ) {
        let pair = vars[ i ].split( '=' );
        if ( pair[0] == key ) {
            return pair[1];
        }
    }
    return false;
}

In [ ]:
/**
 * Generate and automatically download a txt file from the URL contents
 *
 * @param   {string}  contents  The contents to download
 *
 * @return  {void}
 */

#Copy the below code on console and hit enter
function createDownload( contents ) {
    var hiddenElement = document.createElement( 'a' );
    hiddenElement.href = 'data:attachment/text,' + encodeURI( contents );
    hiddenElement.target = '_blank';
    hiddenElement.download = 'urls.txt';
    hiddenElement.click();
}

In [ ]:
/**
 * grab all URLs va a Promise that resolves once all URLs have been
 * acquired
 *
 * @return  {object}  Promise object
 */
    
# Copy the below code on console and hit enter
function grabUrls() {
    var urls = [];
    return new Promise( function( resolve, reject ) {
        var count = document.querySelectorAll(
            '.isv-r a:first-of-type' ).length,
            index = 0;
        Array.prototype.forEach.call( document.querySelectorAll(
            '.isv-r a:first-of-type' ), function( element ) {
            // using the right click menu Google will generate the
            // full-size URL; won't work in Internet Explorer
            // (http://pyimg.co/byukr)
            simulateRightClick( element.querySelector( ':scope img' ) );
            // Wait for it to appear on the <a> element
            var interval = setInterval( function() {
                if ( element.href.trim() !== '' ) {
                    clearInterval( interval );
                    // extract the full-size version of the image
                    let googleUrl = element.href.replace( /.*(\?)/, '$1' ),
                        fullImageUrl = decodeURIComponent(
                            getURLParam( googleUrl, 'imgurl' ) );
                    if ( fullImageUrl !== 'false' ) {
                        urls.push( fullImageUrl );
                    }
                    // sometimes the URL returns a "false" string and
                    // we still want to count those so our Promise
                    // resolves
                    index++;
                    if ( index == ( count - 1 ) ) {
                        resolve( urls );
                    }
                }
            }, 10 );
        } );
    } );
}

In [ ]:
/**
 * Call the main function to grab the URLs and initiate the download
 */

#Copy the below code on console and hit enter 
#This code triggers the url txt file download 
grabUrls().then( function( urls ) {
    urls = urls.join( '\n' );
    createDownload( urls );
} );

## Downloading Google Images using python

In [1]:
# import the necessary packages
from imutils import paths
import argparse
import requests #The request packaage is used to download the image(s) content
import cv2
import os

<b>Load URL downloaded from Javascript codes</b><br>
<b>Loading the urls from the text file into a list and initialising a counter to count the downloaded images</b>

In [17]:
# grab the list of URLs from the input file, then initialize the
# total number of images downloaded thus far

# Directory to URL textfile
rows = open('C:/Users/Student/crime-activity-classification/Normal/urls8.txt').read().strip().split("\n")
total = 0

<b>URLs textfile contains links to all images that needs to be downloaded</b><br>
<b>Looping through all urls in the URL text file and downloading each image</b>

In [18]:
# loop the URLs
for url in rows:
    try:
        # try to download the image
        r = requests.get(url, timeout=60)
        # save the image to disk
        p = os.path.sep.join(['C:/Users/Student/crime-activity-classification/Automated Downloading/Malls', "{}.jpg".format(
            str(total).zfill(8))])
        f = open(p, "wb") #write mode in binary format
        f.write(r.content) #write files content
        f.close()
        # update the counter
        print("INFO: downloaded: {}".format(p))
        total += 1
    # handle if any exceptions are thrown during the download process
    except:
        print("INFO: error downloading {}...skipping".format(p))

INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000000.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000001.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000002.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000003.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000004.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000005.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000006.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000007.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000008.jpg
INFO: downloaded: C:/Users/Student/crime-activ

INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000078.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000079.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000080.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000081.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000082.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000083.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000084.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000085.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000086.jpg
INFO: downloaded: C:/Users/Student/crime-activ

INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000155.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000156.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000157.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000158.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000159.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000160.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000161.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000162.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000163.jpg
INFO: downloaded: C:/Users/Student/crime-activ

INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000233.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000234.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000235.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000236.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000237.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000238.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000239.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000240.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000241.jpg
INFO: downloaded: C:/Users/Student/crime-activ

INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000310.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000311.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000312.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000313.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000314.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000315.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000316.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000317.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000318.jpg
INFO: downloaded: C:/Users/Student/crime-activ

INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000387.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000388.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000389.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000390.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000391.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000392.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000393.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000394.jpg
INFO: downloaded: C:/Users/Student/crime-activity-classification/Automated Downloading/Malls\00000395.jpg


Looping through all files downloaded and attempt opening with OpenCV. If the file cannot be opened with OpenCV it is deleted and the process resumes.

In [26]:
# loop over the image paths we just downloaded
for imagePath in paths.list_images('C:/Users/Student/crime-activity-classification/Automated Downloading/Traffic'):
    # initialize if the image should be deleted or not
    delete = False
    # try to load the image
    try:
        image = cv2.imread(imagePath)
        # if the image is `None` then we could not properly load it
        # from disk, so delete it
        if image is None:
            delete = True
    # if OpenCV cannot load the image then the image is likely
    # corrupt so we should delete it
    except:
        print("Except")
        delete = True
    # check to see if the image should be deleted
    if delete:
        print("[INFO] deleting {}".format(imagePath))
        os.remove(imagePath) #if delete flag is set, os.remove() is called to delete the image

[INFO] deleting C:/Users/Student/crime-activity-classification/Automated Downloading/Traffic\00000028.jpg
[INFO] deleting C:/Users/Student/crime-activity-classification/Automated Downloading/Traffic\00000038.jpg
[INFO] deleting C:/Users/Student/crime-activity-classification/Automated Downloading/Traffic\00000049.jpg
[INFO] deleting C:/Users/Student/crime-activity-classification/Automated Downloading/Traffic\00000052.jpg
[INFO] deleting C:/Users/Student/crime-activity-classification/Automated Downloading/Traffic\00000065.jpg
[INFO] deleting C:/Users/Student/crime-activity-classification/Automated Downloading/Traffic\00000082.jpg
[INFO] deleting C:/Users/Student/crime-activity-classification/Automated Downloading/Traffic\00000173.jpg
[INFO] deleting C:/Users/Student/crime-activity-classification/Automated Downloading/Traffic\00000176.jpg
[INFO] deleting C:/Users/Student/crime-activity-classification/Automated Downloading/Traffic\00000186.jpg
[INFO] deleting C:/Users/Student/crime-activit

<b>Expect images to be corrupt and unable to open: These are deleted from the dataset</b>

<b>Downloading method used</b><br>
Source:https://www.pyimagesearch.com <br>
Title: How to create a deep learning dataset using Google Images 

<b>NB:Pruning irrelevant images from our dataset</b><br>
Not every image downloaded will be (is) relevant. To resolve this manual inspection is required

## Detect and remove duplicate images from a dataset for deep learning

<b>Duplicates in a dataset:</b><br>
1. Introduces bias into the dataset, giving the DNN additional opportunities to learn patterns specific to the duplicates<br>
2. Affects the ability of the model to generalize to new images outside of what it is trained on.

<b>Manually detecting duplicate images in a dataset is extremely time-consuming and error-prone - It does not scale to large image datasets.</b><br>
<b>Therefore a method is needed to automatically detect and remove duplicate images from a dataset.</b>

<b>Possibility of having duplicates:</b> <br>
    
1. Scraping images from multiple sources (e.g.., Google, Bing, etc)
2. Combining existing datasets (ex., combining ImageNet with Sun397 and Indoor Scences)<br><br>
<b>Counter:</b>
1. Detect that there are duplicates images in a dataset
2. Remove the duplicates<br><br>
<b>Why be concerned about duplicates:</b><br>
Assumption for supervised machine learning methods:<br>
1. Data points are independent
2. They are identically distributed 
3. Training and testing data are sampled from the same distribution <br>

The assumptions rarely hold in practice

<b> Note!</b><br>
Identically Distributed means that there are no overall trends–the distribution doesn't fluctuate and all items in the sample are taken from the same probability distribution. Independent means that the sample items are all independent events. In other words, they aren't connected to each other in any way.

### Image duplicate detector

In [27]:
# import the necessary packages
from imutils import paths #to grab the filepaths to all images in the dataset
import numpy as np # for image stacking
import argparse
import cv2 #for image i/o, manipulation, and display
import os

In [28]:
def dhash(image, hashSize=8):
    # convert the image to grayscale and resize the grayscale image,
    # adding a single column (width) so we can compute the horizontal
    # gradient
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (hashSize + 1, hashSize))
    # compute the (relative) horizontal gradient between adjacent
    # column pixels
    diff = resized[:, 1:] > resized[:, :-1]
    # convert the difference image to a hash and return it
    return sum([2 ** i for (i, v) in enumerate(diff.flatten()) if v])

The dhash creates a numerical representation of the image<br>
When two or more images have the same hash, they are considered duplicates.

<b> Test with traffic data: 'C:/Users/Student/crime-activity-classification/Automated Downloading/Traffic/'</b>

In [44]:
# grab the paths to all images in our input dataset directory and
# then initialize our hashes dictionary
print("[INFO] computing image hashes...")
# Directory of the images where duplicates must be removed/view if any exists
imagePaths = list(paths.list_images('C:/Users/Student/crime-activity-classification/CrimeData/data/Weapon/'))
hashes = {}
# loop over our image paths
for imagePath in imagePaths:
    # load the input image and compute the hash
    image = cv2.imread(imagePath)
    h = dhash(image)
    # grab all image paths with that hash, add the current image
    # path to it, and store the list back in the hashes dictionary
    p = hashes.get(h, [])
    p.append(imagePath)
    hashes[h] = p

[INFO] computing image hashes...


<b> Example </b>
{<br>
    ... <br>
    7054210665732718398: ['dataset/00000005.jpg', 'dataset/00000071.jpg', 'dataset/00000869.jpg'],<br>
    8687501631902372966: ['dataset/00000011.jpg'], <br>
    1321903443018050217: ['dataset/00000777.jpg'], <br>
    ... <br>
}

<b>First hash key has three associated image paths and the next two hash keys only have one path entry (no duplicates)</b>

<b>NB: By If 1<= 0:</b><br>
1: Make the 1 value -1 to view duplicate images<br>
2: Leave as 1 to remove the duplicate images

In [46]:
# loop over the image hashes
for (h, hashedPaths) in hashes.items():
    # check to see if there is more than one image with the same hash
    if len(hashedPaths) > 1:
        # check to see if this is a dry run
        if 1 <= 0: #value of -1 to view and value of 1 to remove [e.g. if value <= 0: ]
            # initialize a montage to store all images with the same
            # hash
            montage = None
            # loop over all image paths with the same hash
            for p in hashedPaths:
                # load the input image and resize it to a fixed width
                # and heightG
                image = cv2.imread(p)
                image = cv2.resize(image, (150, 150))
                # if our montage is None, initialize it
                if montage is None:
                    montage = image
                # otherwise, horizontally stack the images
                else:
                    montage = np.hstack([montage, image])
            # show the montage for the hash
            print("[INFO] hash: {}".format(h))
            cv2.imshow("Montage", montage)
            cv2.waitKey(0)
            
# otherwise, we'll be removing the duplicate images
        else:
            # loop over all image paths with the same hash *except*
            # for the first image in the list (since we want to keep
            # one, and only one, of the duplicate images)
            for p in hashedPaths[1:]:
                os.remove(p)

<b>If the --remove flag is not a positive value, we are conducting a dry run</b><br>
<b>Dry run:</b>Viewing the duplicate images 

In [ ]:
# USAGE
# python detect_and_remove.py --dataset dataset: value of -1 to view 
# python detect_and_remove.py --dataset dataset --remove 1: value of 1 to remove

<b>Note:Creating a dataset</b>

Consider a problem and design a data collection for it. 